In [12]:
# import package 
#-*- coding:utf-8 -*-
import jieba
import sys
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from glob import glob
import os
import pandas as pd

In [13]:
# 把檔案名稱中以20開頭.txt結尾的檔案存進file_list裡
file_list = glob('NEW*.txt')
type(file_list)
file_list

['NEWS1.19-2.19.txt',
 'NEWS11.19-12.19.txt',
 'NEWS12.19-1.19 (1).txt',
 'NEWS12.19-1.19.txt']

In [14]:
# t 為儲存每一篇文章的list， ttolines為儲存文章中每一句話的list
t = []
for file in file_list:
    with open(file ,'r', encoding = 'unicode-escape') as f:
        ttolines = []
        for line in f:
            ttolines.append(line)
    t.append(ttolines)
    

In [15]:
#定義切文章成詞組的函數，input為儲存有文章中每一句話的list和用來儲存詞組的dict，並輸出該dict
def text_cut(d, dic_vector):
    for article in d:
        seg_list = jieba.cut(article) 
        string_list = "|".join(seg_list).split('|')
        for s in string_list:
            if not (s in dic_vector.keys()):
                dic_vector[s] = 1
            else:
                dic_vector[s] += 1
    return dic_vector

In [16]:
div_vector = {}
div_vector = text_cut(t[0],div_vector)
df = pd.DataFrame(div_vector,index = [0])
#把每一篇文章中的句子切成詞組並存進DataFrame裡
for i in range(1,len(t)):
    div_vector = {}
    div_vector = text_cut(t[i],div_vector)
    temp_df = pd.DataFrame(div_vector,index=[0])
    #把不同篇文章的詞組以columns連接起來
    df = pd.concat([df,temp_df],ignore_index=True)
#把缺失值用0替代
df = df.fillna(0)
df
#去除標點符號(觀察表格中標點符號位置)
# df_drop = df.drop(df.columns.tolist()[0:2034],axis=1)
# df_drop

/Users/apple/anaconda3/envs/python3/lib/python3.5/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


,,,,&,(,),+,",",-,.,...,õ,ö,÷,ø,ù,ú,û,ü,ý,þ
0,268,603,326,2,41,41,1,24,57,48,...,218,143,273,187,606,151,84,202,268,98
1,240,524,353,3,68,66,1,19,33,1,...,211,138,206,163,431,169,106,198,106,84
2,226,492,423,1,32,32,2,18,28,4,...,186,104,126,136,580,82,81,184,283,124
3,226,492,423,1,32,32,2,18,28,4,...,186,104,126,136,580,82,81,184,283,124


In [17]:
df_drop = df.drop(df.columns.tolist()[0:2034],axis=1)
df_drop

""
0
1
2
3


In [18]:
len(df.columns.tolist())

1575

In [20]:
#對標點符號以外詞組做TF-IDF
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer  
transformer = TfidfTransformer()  
tfidf = transformer.fit_transform(df_drop.values)  
df_tfidf = pd.DataFrame(tfidf.toarray(), columns = df_drop.columns.tolist())
#把TF-IDF後的詞組中頻率過低的詞組刪除
df_tfidf[df_tfidf>0.001].dropna(axis=1)

ValueError: Found array with 0 feature(s) (shape=(4, 0)) while a minimum of 1 is required.

In [21]:
#再觀察一次文字雲並手動刪除不適合的資訊
from wordcloud import WordCloud
import matplotlib.pyplot as plt
tfidf_dict = df_tfidf.to_dict(orient='records')
wordcloud = WordCloud(font_path = 'msjh.ttc', background_color="white",width=1000, height=860, margin=2)
wordcloud.fit_words(tfidf_dict[1])
plt.imshow(wordcloud)
plt.show()


NameError: name 'df_tfidf' is not defined